# IBM Data Science Capstone - Week 3 Assignment
## Segmenting and Clustering Neighborhoods in Toronto

In [74]:
!jupyter trust

Error executing Jupyter command 'trust': [WinError 5] Access is denied


This note book would contains the 3 main sections:
1. Scrape the wikipedia link at https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M to extract the neighborhoods in the GTA area
2. Put the data into an orgainized dataframe
3. Segmenting and Analyzing the Data

### 1. Scrape the data from the wikipedia link:

In [1]:
#import pandas and numpy
import pandas as pd
import numpy as np

#import requests and urlrequest
import requests
import urllib.request
from urllib.request import urlopen

#import BeautifulSoup to scrape html file for data
from bs4 import BeautifulSoup

#import time
import time

In [2]:
#assign the wikipedia link to a variable called url = https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#open the url with the urlopen function and assign the result to a variable called html 
html = urlopen(url)

#Using Beautiful Soup to open the html
soup = BeautifulSoup(html, 'html.parser')

In [3]:
# Check the data type of html and soup
print('the data type of html is {}'.format(type(html)))
print('the data type of soup is {}'.format(type(soup)))


the data type of html is <class 'http.client.HTTPResponse'>
the data type of soup is <class 'bs4.BeautifulSoup'>


In [4]:
#use the find_all function in the Beautiful Soup to find all table tag in the soup variable
tables = soup.find_all('table')

#re-print the data type of tables
print(type(tables))


<class 'bs4.element.ResultSet'>


In [5]:
#Extract the data
#Firstly, we start by creating three empty lists for the three columns in the wikipedia table:
PostalCode = []
Borough = []
Neighborhood = []

In [6]:
#Find all the rows of the first table in the wiki page, as designated as tables[0]
rows = tables[0].find_all('tr')

# Print the number of row in the table
print('The number of row in the wiki table is: {}'.format(len(rows)))

The number of row in the wiki table is: 181


In [7]:
#iterate through all the element in row 
for i in range(1,len(rows)):
    cell = rows[i].find_all('td') #extract the element in each row separating by the td tag
    
    PostalCode.append(cell[0].text.splitlines()[0]) # assign the first element into the PostalCode
    Borough.append(cell[1].text.splitlines()[0]) #assign the 2nd element into Borough
    Neighborhood.append(cell[2].text.splitlines()[0]) #assign the 3rd element into Neighborhood

print(PostalCode[0:5])

['M1A', 'M2A', 'M3A', 'M4A', 'M5A']


In [8]:
#Create a dataframe from the three created list. Name the dataframe GTA
GTA = pd.DataFrame({'PostalCode':PostalCode, 'Borough':Borough, 'Neighborhood':Neighborhood})
GTA.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [9]:
print('The shape of the current GTA dataframe is: ',GTA.shape)

The shape of the current GTA dataframe is:  (180, 3)


### 2. Prepare the Data Frame:
This step involves downloading the geo-coding data into a separate dataframe called GTAGeo then combine that with the original data frame

In [10]:
#download Geocoding data from https://cocl.us/Geospatial_data and import that into GTAGeo Dataframe: 
GTAGeo = pd.read_csv('https://cocl.us/Geospatial_data')
GTAGeo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
GTAGeo.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
GTAGeo.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
#Cleaning the orignal GTA dataframe:
GTA = GTA[GTA['Borough'] != 'Not assigned'].reset_index(drop=True) #Remove row where Borough = 'Not assigned'
GTA['Neighborhood'] = GTA.apply(lambda x: x['Borough'] if x['Neighborhood']=='Not assigned' else x['Neighborhood'], axis=1) 

#if Neighborhood = 'Not assigned' then Neighborhood = Borough
GTA.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [13]:
#Combine GTA and GTAGeo using the merge function with left join
GTA = pd.merge(left=GTA, right=GTAGeo, how='left', left_on='PostalCode', right_on='PostalCode')
GTA.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [14]:
print('The shape of the current GTA dataframe is: ',GTA.shape)

The shape of the current GTA dataframe is:  (103, 5)


### 3. Segmenting and Analyzing the Data

Import sklearn for k-mean, import matplotlib, import folium for mapping

In [15]:
#Import sklearn:
from sklearn.cluster import KMeans

#Import matplotlib
import matplotlib.cm as cm
import matplotlib.colors as color

#import folium for mapping:
!pip install folium 
import folium

#install and import geopy
!pip install geopy
from geopy.geocoders import Nominatim #convert address into longtitude and lattidue

print('Libraries successfully imported')

Libraries successfully imported


Use geocoders to find the longtiude and lattitude of Toronto, ON. Name user_agent to be ca_explorer

In [16]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent='ca_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('The geographical coordinates of {} are {}, {}'.format(address, latitude, longitude))

The geographical coordinates of Toronto, ON are 43.6534817, -79.3839347


Use folium to create a map centered at Toronto, ON, zoom level = 10 with neighborhood superimposed as markers

In [17]:
#create map_GTA with latitude and longitude:
map_GTA = folium.Map(location=[latitude, longitude], zoom_start=10)

#add markers using neighborhood coordidate, one marker is for 1 postal code equivalent to 1 line in the GTA table:
for lat, long, neighborhood, borough in zip(GTA['Latitude'], GTA['Longitude'], GTA['Neighborhood'], GTA['Borough']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat,long],
                       radius=5,
                       popup=label,
                       color='red',
                       fill=True,
                       fill_color='#b06671',
                       fill_opacity=0.7,
                       parse_html=False).add_to(map_GTA)

#show GTA Map
map_GTA

Create a subset of the data for the Borough that contains the word 'Toronto'

In [18]:
Toronto = GTA[GTA['Borough'].str.contains('Toronto')].reset_index(drop=True)
Toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [19]:
#Print the shape of Toronto subset
print('The shape of the Toronto subset is {}'.format(Toronto.shape))

The shape of the Toronto subset is (39, 5)


Use folium to create a map of Toronto Borough:

In [20]:
#create map_Toronto with latitude and longitude:
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

#add markers using neighborhood coordidate, one marker is for 1 postal code equivalent to 1 line in theToronto table:
for lat, long, neighborhood, borough in zip(Toronto['Latitude'], Toronto['Longitude'], Toronto['Neighborhood'], Toronto['Borough']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat,long],
                       radius=5,
                       popup=label,
                       color='red',
                       fill=True,
                       fill_color='#b06671',
                       fill_opacity=0.7,
                       parse_html=False).add_to(map_Toronto)

#show Toronto Map
map_Toronto

Define Foursquare Credentials and Version:

In [21]:
client_id = 'CATERQLL2NYX0OE2YSMHCII0VUHNQL1Q3LGIYYESQAS5HIUQ'
client_secret = 'PERDT3Q0I5L5AXOSSJH1T3S0JZ2SJFUPGOFOO0JUQ3KBYTLR'
Access_token = '3SGO4VKYCVHL2BC4M0VOOCGBUJBT32GHU4DCKD2HFTNBTN1C'
version = '20180605' #Foursquare API version
radius = 500
limit = 100# default Foursquare API limit

#print credentials:
print('Your credentials:')
print('Client_ID: '+client_id)
print('Client_secret: '+client_secret)

Your credentials:
Client_ID: CATERQLL2NYX0OE2YSMHCII0VUHNQL1Q3LGIYYESQAS5HIUQ
Client_secret: PERDT3Q0I5L5AXOSSJH1T3S0JZ2SJFUPGOFOO0JUQ3KBYTLR


#### a) Explore the downtown Toronto with latitude and longitude as the first row of the Toronto table

In [22]:
borough_latitude = Toronto.loc[0,'Latitude']
borough_longitude = Toronto.loc[0,'Longitude']

borough_name = Toronto.loc[0,'Borough']
print('Latitude and Longitude of {} are {},{}'.format(borough_name, borough_latitude, borough_longitude))

Latitude and Longitude of Downtown Toronto are 43.6542599,-79.3606359


Create an URL to access Foursquare API with the client information above:

In [23]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&&v={}&radius={}&limit{}'.format(
client_id,client_secret,borough_latitude,borough_longitude,version,radius,limit)
print(url)

https://api.foursquare.com/v2/venues/explore?client_id=CATERQLL2NYX0OE2YSMHCII0VUHNQL1Q3LGIYYESQAS5HIUQ&client_secret=PERDT3Q0I5L5AXOSSJH1T3S0JZ2SJFUPGOFOO0JUQ3KBYTLR&ll=43.6542599,-79.3606359&&v=20180605&radius=500&limit100


Send a GET request and import the data into results variable:

In [24]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fcae8b66ef0be321d73ad90'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 44,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
 

Get relevant part of JSON and transform it into a pandas dataframe


In [25]:
#assign relevant part of JSON to venues:
venues = results['response']['groups'][0]['items']

#import json_normalize:
from pandas.io.json import json_normalize

#transform venues into a dataframe
df = json_normalize(venues)
df.head()


<ipython-input-25-7adeb2a26327>:8: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(venues)


,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.crossStreet,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,...,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.venuePage.id,venue.location.neighborhood
0,e-0-54ea41ad498e9a11e9e13308-0,0,"[{'summary': 'This spot is popular', 'type': '...",54ea41ad498e9a11e9e13308,Roselle Desserts,362 King St E,Trinity St,43.653447,-79.362017,"[{'label': 'display', 'lat': 43.65344672305267...",...,CA,Toronto,ON,Canada,"[362 King St E (Trinity St), Toronto ON M5A 1K...","[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",0,[],NaN,NaN
1,e-0-53b8466a498e83df908c3f21-1,0,"[{'summary': 'This spot is popular', 'type': '...",53b8466a498e83df908c3f21,Tandem Coffee,368 King St E,at Trinity St,43.653559,-79.361809,"[{'label': 'display', 'lat': 43.65355870959944...",...,CA,Toronto,ON,Canada,"[368 King St E (at Trinity St), Toronto ON, Ca...","[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",0,[],NaN,NaN
2,e-0-574c229e498ebb5c6b257902-2,0,"[{'summary': 'This spot is popular', 'type': '...",574c229e498ebb5c6b257902,Cooper Koo Family YMCA,461 Cherry St,NaN,43.653249,-79.358008,"[{'label': 'display', 'lat': 43.65324910177244...",...,CA,Toronto,ON,Canada,"[461 Cherry St, Toronto ON M5A 0H7, Canada]","[{'id': '52e81612bcbc57f1066b7a37', 'name': 'D...",0,[],NaN,NaN
3,e-0-50760559e4b0e8c7babe2497-3,0,"[{'summary': 'This spot is popular', 'type': '...",50760559e4b0e8c7babe2497,Body Blitz Spa East,497 King Street East,btwn Sackville St and Sumach St,43.654735,-79.359874,"[{'label': 'display', 'lat': 43.65473505045365...",...,CA,Toronto,ON,Canada,[497 King Street East (btwn Sackville St and S...,"[{'id': '4bf58dd8d48988d1ed941735', 'name': 'S...",0,[],NaN,NaN
4,e-0-5612b1cc498e3dd742af0dc8-4,0,"[{'summary': 'This spot is popular', 'type': '...",5612b1cc498e3dd742af0dc8,Impact Kitchen,573 King St E,at St Lawrence St,43.656369,-79.356980,"[{'label': 'display', 'lat': 43.65636850543279...",...,CA,Toronto,ON,Canada,"[573 King St E (at St Lawrence St), Toronto ON...","[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",0,[],NaN,NaN


Definite a get_category_type(row) function

In [26]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Clean the newly created dataframe df

In [27]:
#filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = df.loc[:,filtered_columns]

#filter the category for each row:
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

#clean columns:
nearby_venues.columns = [col.split('.')[-1] for col in nearby_venues.columns]
nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Impact Kitchen,Restaurant,43.656369,-79.356980


Return the number of venues from Foursquare:

In [28]:
print('The number of venues from Foursquare for {}, {} is {}'.format(borough_latitude, borough_longitude, nearby_venues.shape[0]))

The number of venues from Foursquare for 43.6542599, -79.3606359 is 30


#### b) Create a function to repeat the step for all latitude, longtitude in the Toronto dataframe:

In [29]:
def getNearbyVenues(names, latitudes, longitudes, radius = 500):
    
    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        #create API request URL for Foursquare
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        client_id,
        client_secret,
        version,
        lat,
        lng,
        radius,
        limit)
        
        #make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        #return only relevant information:
        venues_list.append([(
        name,
        lat,
        lng,
        v['venue']['name'],
        v['venue']['location']['lat'],
        v['venue']['location']['lng'],
        v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues_list = ([item for venue_list in venues_list for item in venue_list])
    nearby_venues = pd.DataFrame(nearby_venues_list)
    nearby_venues.columns = ['Neighborhood',
                            'Neighborhood Latitude',
                            'Neighborhood Longitude',
                            'Venue',
                            'Venue Latitude',
                            'Venue Longitude',
                            'Venue Category']

    return(nearby_venues)

#### c) Apply the defined getNearbyVenues function to all the rows in the Toronto dataframe:

In [30]:
Toronto_venues = getNearbyVenues(names=Toronto['Neighborhood'],latitudes=Toronto['Latitude'], longitudes=Toronto['Longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport


In [32]:
#Examine the Toronto_venues dataframe
print('The number of venues returned from Foursquare for Toronto dataframe is {}'.format(Toronto_venues.shape))
Toronto_venues.head()

The number of venues returned from Foursquare for Toronto dataframe is (1624, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


Check the number of venues for each neighbhorhood:

In [33]:
Toronto_venues[['Neighborhood','Venue']].groupby('Neighborhood').count().sort_values(['Venue'],ascending=False)

,Venue
Neighborhood,
"Garden District, Ryerson",100
"Toronto Dominion Centre, Design Exchange",100
"Commerce Court, Victoria Hotel",100
"Richmond, Adelaide, King",100
"First Canadian Place, Underground city",100
"Harbourfront East, Union Station, Toronto Islands",100
Stn A PO Boxes,96
St. James Town,85
Church and Wellesley,75


Return the number of unique categories from all the venues:

In [34]:
print('There are {} uniques categories'.format(len(Toronto_venues['Venue Category'].unique())))

There are 235 uniques categories


#### d) Analyze each neighborhood:

In [35]:
#one hot ecoding:
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix='', prefix_sep='')

#add neighborhood column to to the dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood']

#fixed column:
column_list = list(Toronto_onehot.columns)
column_list.remove('Neighborhood')
column_list = ['Neighborhood'] + column_list

Toronto_onehot = Toronto_onehot[column_list]
Toronto_onehot.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Groupby neighborhood and by taking the mean of the frequency of occurance of each category

In [36]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped


,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.018182,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0625,0.0625,0.0625,0.125,0.125,0.0625,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.014706,0.000000,0.000000,0.014706,0.014706
5,Christie,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.013333,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.013333,0.000000,...,0.013333,0.013333,0.000000,0.000000,0.00,0.000000,0.000000,0.013333,0.000000,0.026667
7,"Commerce Court, Victoria Hotel",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.030000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000
8,Davisville,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,...,0.000000,0.000000,0.030303,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000


In [37]:
print(Toronto_grouped.shape) #Confirm the sahpe of the new size

(39, 235)


Print each neighborhood along with the top 5 most common venues

In [38]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+'----')
    temp = Toronto_grouped[Toronto_grouped['Neighborhood']==hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq',ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


----Berczy Park----
            venue  freq
0     Coffee Shop  0.09
1     Cheese Shop  0.04
2      Restaurant  0.04
3          Bakery  0.04
4  Farmers Market  0.04


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.13
1       Nightclub  0.09
2     Coffee Shop  0.09
3  Breakfast Spot  0.09
4             Gym  0.04


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
           venue  freq
0           Park  0.06
1  Auto Workshop  0.06
2        Brewery  0.06
3     Skate Park  0.06
4  Burrito Place  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12
2   Harbor / Marina  0.06
3             Plane  0.06
4  Sculpture Garden  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.18
1                Café  0.06
2      Sandwich Pl

Put the results into a pandas DataFrame

In [39]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=True)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [40]:
num_top_values = 5

indicators = ['st', 'nd', 'rd']

#create columns according to number of top venues:
columns = ['Neighborhood']
for ind in np.arange(num_top_values):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{} Most Common Venue'.format(ind+1))

#create new dataframe
neighborhood_venues_sorted = pd.DataFrame(columns=columns)
neighborhood_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhood_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind,:], num_top_values)
    
neighborhood_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4 Most Common Venue,5 Most Common Venue
0,Berczy Park,Afghan Restaurant,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake
1,"Brockton, Parkdale Village, Exhibition Place",Afghan Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge
2,"Business reply mail Processing Centre, South C...",Afghan Restaurant,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge
3,"CN Tower, King and Spadina, Railway Lands, Har...",Afghan Restaurant,Lingerie Store,Liquor Store,Lounge,Malay Restaurant
4,Central Bay Street,Afghan Restaurant,Jazz Club,Jewelry Store,Kitchen Supply Store,Lake


#### e) cluster downtown Toronto into different clusters

In [64]:
#set number of cluster = 5
k = 5
Toronto_grouped_cluster = Toronto_grouped.drop('Neighborhood',1)

#run k-mean cluterin:
kmeans = KMeans(n_clusters=k,random_state=0).fit(Toronto_grouped_cluster)

#check cluster labels generated for each row in the grame. 
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

  Create new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [65]:
#add cluster label:
try:
    neighborhood_venues_sorted.insert(0,'Cluster labels', kmeans.labels_)
except ValueError:
    neighborhood_venues_sorted['Cluster labels'] = kmeans.labels_

Toronto_merged = Toronto

#merge Toront_grouped with Toronto_data to add lattitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhood_venues_sorted.set_index('Neighborhood'),on='Neighborhood')

Toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster labels,Cluster label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4 Most Common Venue,5 Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,0,Afghan Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,0,Afghan Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,0,Afghan Restaurant,Hostel,Hotel Bar,IT Services,Ice Cream Shop
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,0,Afghan Restaurant,Juice Bar,Korean Restaurant,Lake,Light Rail Station
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,0,Afghan Restaurant,Lingerie Store,Liquor Store,Lounge,Malay Restaurant


Visualize cluster on a map

In [63]:
#import colors
import matplotlib.colors as colors

#create map
cluster_map = folium.Map(location=[latitude, longitude], zoom_start=11)

#set color scheme for the clusters:
x = np.arange(k)
ys = [1+x+(i*x)**2 for i in range(k)]
color_array = cm.rainbow(np.linspace(0,1,len(ys)))
rainbow = [colors.rgb2hex(i) for i in color_array]

#add marker to the map:
marker_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster labels']):
    label = folium.Popup(str(poi) + 'Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(cluster_map)

cluster_map

Examine the clusters:

1. Cluster 1

In [66]:
Toronto_merged.loc[Toronto_merged['Cluster labels']==0, Toronto_merged.columns[[1]+list(range(5,Toronto_merged.shape[1]))]]

,Borough,Cluster labels,Cluster label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4 Most Common Venue,5 Most Common Venue
0,Downtown Toronto,0,0,Afghan Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store
1,Downtown Toronto,0,0,Afghan Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store
2,Downtown Toronto,0,0,Afghan Restaurant,Hostel,Hotel Bar,IT Services,Ice Cream Shop
3,Downtown Toronto,0,0,Afghan Restaurant,Juice Bar,Korean Restaurant,Lake,Light Rail Station
4,East Toronto,0,0,Afghan Restaurant,Lingerie Store,Liquor Store,Lounge,Malay Restaurant
5,Downtown Toronto,0,0,Afghan Restaurant,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake
6,Downtown Toronto,0,0,Afghan Restaurant,Jazz Club,Jewelry Store,Kitchen Supply Store,Lake
7,Downtown Toronto,0,0,Afghan Restaurant,Lingerie Store,Liquor Store,Lounge,Malay Restaurant
8,Downtown Toronto,0,0,Afghan Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant
9,West Toronto,0,0,Afghan Restaurant,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store


2. Cluster 2

In [67]:
Toronto_merged.loc[Toronto_merged['Cluster labels']==1, Toronto_merged.columns[[1]+list(range(5,Toronto_merged.shape[1]))]]

,Borough,Cluster labels,Cluster label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4 Most Common Venue,5 Most Common Venue
29,Central Toronto,1,1,Afghan Restaurant,Lingerie Store,Liquor Store,Lounge,Malay Restaurant


3. Cluster 3

In [68]:
Toronto_merged.loc[Toronto_merged['Cluster labels']==2, Toronto_merged.columns[[1]+list(range(5,Toronto_merged.shape[1]))]]

,Borough,Cluster labels,Cluster label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4 Most Common Venue,5 Most Common Venue
18,Central Toronto,2,2,Afghan Restaurant,Lingerie Store,Liquor Store,Lounge,Malay Restaurant


4. Cluster 4

In [69]:
Toronto_merged.loc[Toronto_merged['Cluster labels']==3, Toronto_merged.columns[[1]+list(range(5,Toronto_merged.shape[1]))]]

,Borough,Cluster labels,Cluster label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4 Most Common Venue,5 Most Common Venue
21,Central Toronto,3,3,Afghan Restaurant,Lingerie Store,Liquor Store,Lounge,Malay Restaurant
33,Downtown Toronto,3,3,Afghan Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge


5. Cluster 5

In [70]:
Toronto_merged.loc[Toronto_merged['Cluster labels']==4, Toronto_merged.columns[[1]+list(range(5,Toronto_merged.shape[1]))]]

,Borough,Cluster labels,Cluster label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4 Most Common Venue,5 Most Common Venue
19,Central Toronto,4,4,Afghan Restaurant,Lingerie Store,Liquor Store,Lounge,Malay Restaurant
